## Object recognition with YOLO##


In [1]:
import os
import torch
import cv2
import pandas as pd
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Function to get user-defined folder paths
def get_folder_path(prompt, create_if_missing=False):
    while True:
        path = input(prompt).strip()
        if path:
            if os.path.exists(path) or create_if_missing:
                os.makedirs(path, exist_ok=True)  # Create folder if missing
                print(f"✅ Using folder: {path}")
                return path
        print("❌ Invalid or empty path. Please enter a valid folder path.")

# Function to list available folders before selection
def list_available_folders(base_path):
    print("📂 Available folders:")
    folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]
    if not folders:
        print("❌ No folders found. Exiting.")
        exit()
    for idx, folder in enumerate(folders, start=1):
        print(f"{idx}. {folder}")
    return folders

# Load YOLO model (using YOLOv8 pretrained model)
def load_yolo_model():
    print("🔄 Loading YOLOv8 model...")
    model = YOLO("yolov8n.pt")  # Using YOLOv8 Nano model (lightweight)
    print("✅ YOLO model loaded successfully!")
    return model

# Perform object detection on an image
def detect_objects(model, image_path):
    image = cv2.imread(image_path)
    results = model(image)  # Perform detection
    detections = []
    
    for result in results:
        for box in result.boxes.data:
            x1, y1, x2, y2, conf, cls = box.tolist()
            detections.append({
                "image": os.path.basename(image_path),
                "class": model.names[int(cls)],
                "confidence": round(conf, 2),
                "bbox": [int(x1), int(y1), int(x2), int(y2)]
            })
    return detections

# Visualize detections and save annotated images
def visualize_and_save(image_path, detections, output_folder):
    image = cv2.imread(image_path)
    for det in detections:
        x1, y1, x2, y2 = det["bbox"]
        label = f"{det['class']} ({det['confidence']})"
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    output_path = os.path.join(output_folder, os.path.basename(image_path))
    cv2.imwrite(output_path, image)
    print(f"✅ Saved annotated image: {output_path}")

# Function to manually select which movies to process
def select_movies(movies):
    print("📂 Available movie folders:")
    for idx, movie in enumerate(movies, start=1):
        print(f"{idx}. {movie}")
    
    selected_indices = input("Enter the numbers of the movies you want to analyze (comma-separated, e.g., 1,3,5): ")
    selected_indices = [int(i.strip()) - 1 for i in selected_indices.split(',') if i.strip().isdigit() and 0 <= int(i.strip()) - 1 < len(movies)]
    
    if not selected_indices:
        print("❌ No valid selections made. Exiting.")
        exit()
    
    selected_movies = [movies[i] for i in selected_indices]
    return selected_movies

# Function to process all selected movies
def process_selected_movies(model, keyframe_base_folder, output_csv, output_image_folder):
    movies = list_available_folders(keyframe_base_folder)
    selected_movies = select_movies(movies)
    
    detections_list = []
    
    for movie in selected_movies:
        movie_path = os.path.join(keyframe_base_folder, movie)
        print(f"📂 Processing keyframes from movie: {movie}")
        
        keyframe_images = [f for f in os.listdir(movie_path) if f.endswith(".jpg")]
        for img in keyframe_images:
            img_path = os.path.join(movie_path, img)
            detections = detect_objects(model, img_path)
            detections_list.extend(detections)
            visualize_and_save(img_path, detections, output_image_folder)
    
    # Save detections to CSV
    df = pd.DataFrame(detections_list)
    df.to_csv(output_csv, index=False)
    print(f"✅ Object detection results saved to {output_csv}")

# Main execution
if __name__ == "__main__":
    keyframe_base_folder = get_folder_path("Enter the base folder containing movie keyframes: ")
    output_csv = os.path.join(keyframe_base_folder, "object_detection_results.csv")
    output_image_folder = get_folder_path("Enter the folder to save annotated images: ", create_if_missing=True)
    
    yolo_model = load_yolo_model()
    process_selected_movies(yolo_model, keyframe_base_folder, output_csv, output_image_folder)
    
    print("✅ Object detection process completed!")


✅ Using folder: /Users/alicja/Desktop/VideoAnalyse/Corpus/KeyFrames/ActionKeyframes
✅ Using folder: /Users/alicja/Desktop/VideoAnalyse/Corpus/ObjectRecognition/ActionRecognition/Action_01
🔄 Loading YOLOv8 model...
✅ YOLO model loaded successfully!
📂 Available folders:
1. Action_01
2. Action_02
3. Action_05
4. Action_04
5. Action_03
📂 Available movie folders:
1. Action_01
2. Action_02
3. Action_05
4. Action_04
5. Action_03
📂 Processing keyframes from movie: Action_01

0: 384x640 (no detections), 90.9ms
Speed: 5.0ms preprocess, 90.9ms inference, 5.7ms postprocess per image at shape (1, 3, 384, 640)
✅ Saved annotated image: /Users/alicja/Desktop/VideoAnalyse/Corpus/ObjectRecognition/ActionRecognition/Action_01/Action_01_RETRIBUTION｜  Action Short film_scene_3_keyframe.jpg

0: 384x640 (no detections), 56.6ms
Speed: 1.2ms preprocess, 56.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
✅ Saved annotated image: /Users/alicja/Desktop/VideoAnalyse/Corpus/ObjectRecognition/Ac